In [ ]:
import operator
from collections import defaultdict
import numpy as np
def parseTrainTestFile(train_marks_filename, isTest=False):

    trainDidToLine = {}
    qidToDocs = defaultdict(list)
    with open(train_marks_filename) as f:
        i=0
        nlines = 0
        for line in f:
            nlines+=1
        
       
        qids = np.zeros( nlines, dtype=int)
        i=0
        f.seek(0)
        if isTest:
            f.readline()
        for ii,line in enumerate(f):
            if not isTest:
                parts = line.split("\t")
                if len(parts)!=3:
                    continue
                qid, did, mark = parts
                qid, did, mark = int(qid), int(did), int(mark)
            else:
                parts = line.split(",")
                if len(parts)!=2:
                    continue
                qid, did = parts
                mark=-1
                qid, did, mark = int(qid), int(did), int(mark)

            qids[i] = qid
            trainDidToLine[(qid,did)] = i

            qidToDocs[qid].append(did)
            i+=1
    return qids, trainDidToLine, qidToDocs
q_train, trainDidToLine, trainQidToDocs = parseTrainTestFile("train.marks.tsv", 
                                                    isTest=False, 
                                                    )

q_test, testDidToLine, testQidToDocs = parseTrainTestFile("sample.csv", 
                                                    isTest=True, 
                                                    )

In [ ]:
from rank_bm25 import BM25Okapi, BM25Plus, BM25L


In [ ]:
bms = [BM25Okapi, BM25Plus, BM25L]

In [ ]:
titles_corpus = []
dids = []
didToLine = {}
with open("../data/normal_titles.txt") as f:
    for i,line in enumerate(f):
        line = line.strip()
        did,title = line.split("\t")
        did = int(did)
        words = title.split(" ")
        titles_corpus.append(words)
        dids.append(did)
        didToLine[did] = i

In [ ]:
queries_corpus = []
qids = []
qidToLine = {}
with open("../data/normal_queries.txt") as f:
    for line in f:
        line = line.strip()
        qid,title = line.split("\t")
        qid = int(qid)
        words = title.split(" ")
        queries_corpus.append(words)
        qidToLine[qid] = len(qids)
        qids.append(qid)
        

In [ ]:
titles_corpus=[]

In [ ]:
for bmid,bm in enumerate(bms):
    bm25 = bm(titles_corpus)
    with open("train_bm25_" + str(bmid) + ".txt", "w") as f:
        #qidToDocsScores = defaultdict(list)
        cnt=0
        for qid,dids in trainQidToDocs.items():
            scores = bm25.get_scores(queries_corpus[qidToLine[qid]])
            for did in dids:
                if did in didToLine:
                    did_line = didToLine[did]
                    #qidToDocsScores[qid].append( (did,scores[did_line]))
                    f.write(str(qid) + "," + str(did) + "," + str(scores[did_line]) + "\n")

            cnt+=1
            if(cnt%200==0):
                print(cnt)

In [ ]:
for bmid,bm in enumerate(bms):
    bm25 = bm(titles_corpus)
    with open("test_bm25_" + str(bmid) + ".txt", "w") as f:
        #qidToDocsScores = defaultdict(list)
        cnt=0
        for qid,dids in testQidToDocs.items():
            scores = bm25.get_scores(queries_corpus[qidToLine[qid]])
            for did in dids:
                  if did in didToLine:
                    did_line = didToLine[did]

                    #qidToDocsScores[qid].append( (did,scores[did_line]))
                    f.write(str(qid) + "," + str(did) + "," + str(scores[did_line]) + "\n")

            cnt+=1
            if(cnt%200==0):
                print(cnt)

In [ ]:
def url_normalize(url):
    url = url.strip()
    url = url.replace("https://", "")
    url = url.replace("http://", "")
    if len(url)>1:
        if url[-1] == "/":
            url = url[:-1]
        url = url.lower()
        url = url.replace(":80", "")
        url = url.replace("www.", "")
        url = url.replace("//", "")
    return url
def extract_domain(url):
    global rubrics_list
    url = url.strip()
    #for r in rubrics_list:
        #url = url.replace("," + r + "," , "")    
    #    url = url.replace("," + r, "") 
    url = url.replace("https://", "")
    url = url.replace("http://", "")
   
    url = url.replace("ftp://", "")
    if len(url)>1:
        if url[-1] == "/":
            url = url[:-1]
        url = url.lower()
        
        url = url.replace("s://", "")
        url = url.replace(":80", "")
        url = url.replace("://", "")
        url = url.replace("www.", "")
        url = url.replace("//", "")
        pos = url.find("#")
        if pos>0:
            url = url[0:pos]

        pos = url.find("/")
        if pos>0:
            url = url[0:pos]
    return url
def loadUrls(filename):
    urlToDid = {}
    didToUrl = {}
    domainToDid = defaultdict(list)
    didToDomain = defaultdict(list)
   
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            uid, url = line.split("\t")
            uid = int(uid)
            #urls["http://" + url] = uid
            #urls["http://" + url + "/"] = uid
            urlToDid[url_normalize(url)] = uid
            didToUrl[uid] = url_normalize(url)
            domain = extract_domain(url)
            domainToDid[domain].append(uid)
            didToDomain[uid] = domain
            
    return urlToDid,didToUrl,domainToDid,didToDomain
urlToDid,didToUrl,domainToDid,didToDomain = loadUrls("url.data")
urls = urlToDid

In [ ]:
cnt=0
not_matched = 0
nqueries=0
def tmp():
    return [10000,10000]
cnt=0
not_matched = 0
nqueries=0

def loadTopDocsFile(filename):
    found_docs = {}
    with open(filename, "r") as f:
        for line in f:
            line = line.strip()
            qid,fd = line.split(",")
            qid = int(qid)
            fd = int(fd)
            found_docs[qid] = (fd,0.0)
    return found_docs
found_docs = loadTopDocsFile("first_top_docs.txt")
found_domains = loadTopDocsFile("first_top_domains.txt")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def titlesGen():
   
    for i,t in enumerate(titles_corpus):
        s = ""
        for w in t:
            s+=w + " "
        yield s
        if i% (len(titles_corpus)//100) == 0:
            print( i/len(titles_corpus))
   

In [ ]:
vectorizer = TfidfVectorizer(stop_words=['english'], analyzer = "char", ngram_range=(2,6))

In [ ]:
titles_tfidf = vectorizer.fit_transform(titlesGen())

In [ ]:
def queriesGen():
   
    for i,t in enumerate(queries_corpus):
        s = ""
        for w in t:
            s+=w + " "
        yield s
        if i% (len(queries_corpus)//100) == 0:
            print( i/len(queries_corpus))
   
queries_tfidf = vectorizer.transform(queriesGen())

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
with open("train_tdidf_ngrams_w.txt", "w") as f:
    
    cnt=0
    for qid,dids in trainQidToDocs.items():
       
        for did in dids:
            if did in didToLine:
                did_line = didToLine[did]
                score = cosine_similarity(queries_tfidf[qidToLine[qid]], titles_tfidf[did_line] )

                #qidToDocsScores[qid].append( (did,scores[did_line]))
                f.write(str(qid) + "," + str(did) + "," + str(score[0][0]) + "\n")

        cnt+=1
        if(cnt%200==0):
            print(cnt)

In [ ]:
with open("test_tdidf_ngrams_w.txt", "w") as f:
    #qidToDocsScores = defaultdict(list)
    cnt=0
    for qid,dids in testQidToDocs.items():
       
        for did in dids:
            if did in didToLine:
                did_line = didToLine[did]
                score = cosine_similarity(queries_tfidf[qidToLine[qid]], titles_tfidf[did_line] )

                #qidToDocsScores[qid].append( (did,scores[did_line]))
                f.write(str(qid) + "," + str(did) + "," + str(score[0][0]) + "\n")

        cnt+=1
        if(cnt%200==0):
            print(cnt)

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

In [ ]:

from nltk.corpus import stopwords
import nltk
#nltk.download('stopwords')
stopWords = list(stopwords.words('russian')) + list(stopwords.words('english'))
vectorizer = HashingVectorizer(stop_words=stopWords)

In [ ]:
stopWords

In [ ]:
didtoline = []
import os

def docsGen2():
    doc_files = os.listdir("../normalize_docs/")
    doc_files = list(map(lambda x: "../normalize_docs/"+x, docs))
    global didtoline
    didtoline = []
    cnt=0
    for filename in doc_files:
        with open(filename) as f:
            for line in f:
                line = line.strip()
                
                parts = line.split("\t", maxsplit=1)
                if len(parts)==2:
                    did = int(parts[0])
                    didtoline.append(did)
                    content = parts[1]
                    yield did,content
                    if(cnt%1000==0):
                        print(cnt)
                    cnt+=1